In [2]:
import pandas as pd
from geopy.geocoders import Nominatim, GoogleV3
from geopy.extra.rate_limiter import RateLimiter

with open('chave.txt') as f:
    chave = f.readlines()[0]


df_cia = pd.read_excel('raw/Consulta CIA_Candeias_Simoes Filho.xlsx', dtype=str)

    # Columns from end_lougradouro to end_cep fillna with ''
df_cia['end_logradouro'] = df_cia['end_logradouro'].fillna('')
df_cia['end_numero'] = df_cia['end_numero'].fillna('')
df_cia['end_complemento'] = df_cia['end_complemento'].fillna('')
df_cia['end_bairro'] = df_cia['end_bairro'].fillna('')
df_cia['mun_descricao'] = df_cia['mun_descricao'].fillna('')
df_cia['end_uf'] = df_cia['end_uf'].fillna('')
df_cia['end_cep'] = df_cia['end_cep'].fillna('')


df_cia['Endereço'] = df_cia['end_logradouro'] + ', ' + df_cia['end_numero'] + ', ' + df_cia['end_complemento'] + ', ' + df_cia['end_bairro'] + ', ' + df_cia['mun_descricao'] + ', ' + df_cia['end_uf']
df_cia['Endereço_sem_bairro'] = df_cia['end_logradouro'] + ', ' + df_cia['end_numero'] + ', ' + df_cia['end_complemento'] + ', ' + df_cia['mun_descricao'] + ', ' + df_cia['end_uf']
df_cia['Endereço'] = df_cia['Endereço'].str.replace('(, )+', ', ', regex=True)
df_cia['Endereço_sem_bairro'] = df_cia['Endereço_sem_bairro'].str.replace('(, )+', ', ', regex=True)

df_cia.head()

,cnpj,razao_social,nome_fantasia,end_logradouro,end_numero,end_complemento,end_bairro,mun_descricao,end_uf,end_cep,...,cnae_denominacao,pri_produto,e-mail,site,principal_executivo,cargo,cadastro,atualizacao,Endereço,Endereço_sem_bairro
0,02.119.258/0002-18,Fertipar Fertilizantes do Nordeste Ltda,Fertipar,Via Matoim,S/N,CIA Norte,Distrito Industrial,Candeias,BA,43813000,...,"Fabricação de adubos e fertilizantes, exceto o...",Adubo e fertilizante,controladoria@fertipar.com.br,www.fertipar.com.br,Alceu Feldmann,Presidente,2000-08-21 00:00:00,2023-03-21 00:00:00,"Via Matoim, S/N, CIA Norte, Distrito Industria...","Via Matoim, S/N, CIA Norte, Candeias, BA"
1,01.002.047/0003-19,Plumatex Colchoes Industrial Ltda,Plumatex,Rod BA-522 Km 01,S/N,,Distrito Industrial,Candeias,BA,43813000,...,Fabricação de colchões,"Colchao de espuma, Colchao de mola, Lamina pol...",recepcaoba@plumatex.com.br,www.plumatex.com.br,Rafael Miguel de Melo,Diretor Comercial,2000-08-22 00:00:00,2022-10-14 00:00:00,"Rod BA-522 Km 01, S/N, Distrito Industrial, Ca...","Rod BA-522 Km 01, S/N, Candeias, BA"
2,61.156.501/0086-45,Mosaic Fertilizantes do Brasil Ltda,Mosaic,Rod BA-522 Km 01,S/N,Cx Postal 22,Distrito Industrial,Candeias,BA,43813000,...,"Fabricação de adubos e fertilizantes, exceto o...","Adubo e fertilizante (NPK), Adubo e fertilizan...",NaN,www.mosaicco.com.br,Floris Eduard Alexsander Bielders,Presidente,2000-08-22 00:00:00,2022-09-19 00:00:00,"Rod BA-522 Km 01, S/N, Cx Postal 22, Distrito ...","Rod BA-522 Km 01, S/N, Cx Postal 22, Candeias, BA"
3,27.515.154/0012-25,Proquigel Quimica S/A,Policarbonatos,Via Matoim,S/N,Fazenda Caroba,Distrito Industrial,Candeias,BA,43813000,...,Fabricação de intermediários para plastificant...,"Metacrilato de metila (MMA), Metacrilato de et...",mauro.duarte@unigel.com.br,www.unigel.com.br,Roberto Noronha Santos,Presidente,2000-10-04 00:00:00,2023-11-22 00:00:00,"Via Matoim, S/N, Fazenda Caroba, Distrito Indu...","Via Matoim, S/N, Fazenda Caroba, Candeias, BA"
4,02.402.478/0001-73,Unigel Plasticos S/A,Unigel,Via Matoim,S/N,Fazenda Caroba,Distrito Industrial,Candeias,BA,43813000,...,Fabricação de artefatos de material plástico p...,"Chapa acrilica, Chapa acrilica extrusada, Resi...",mauro.duarte@unigel.com.br,www.unigel.com.br,Roberto Noronha Santos,Presidente,2000-10-04 00:00:00,2023-07-14 00:00:00,"Via Matoim, S/N, Fazenda Caroba, Distrito Indu...","Via Matoim, S/N, Fazenda Caroba, Candeias, BA"


In [8]:
def faz_geolocalizacao_google(df_cia):
    geolocator = GoogleV3(api_key=chave)
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    df_cia['geolocalizacao'] = df_cia['Endereço'].apply(geocode)
    df_cia['latitude'] = df_cia['geolocalizacao'].apply(lambda x: x.latitude)
    df_cia['longitude'] = df_cia['geolocalizacao'].apply(lambda x: x.longitude)

    df_cia.to_csv('dados/dados_cia_geolocalizados.csv', sep=';', index=False)
    return df_cia[['geolocalizacao', 'latitude', 'longitude']]

# faz_geolocalizacao_google(df_cia)

0      (Distrito Industrial, Candeias - BA, 43820-270...
1      (BA-522, Bahia, Brazil, (-12.6678095, -38.5497...
2      (BA-522, Bahia, Brazil, (-12.6678095, -38.5497...
3      (R. do Passé, 22 - Centro, Candeias - BA, 4380...
4      (R. do Passé, 22 - Centro, Candeias - BA, 4380...
                             ...                        
185    (R. Vivaldo Pain, 1 - 1090, Simões Filho - BA,...
186    (R. Cidade de Belmonte, 834, Salvador - BA, 43...
187    (Vila CIA Sul, Simões Filho - BA, 43700-000, B...
188    (Vila CIA Sul, Simões Filho - BA, 43700-000, B...
189    (Simoes Filho, Simões Filho - State of Bahia, ...
Name: geolocalizacao, Length: 190, dtype: object

# Teste com Nominatim

In [5]:
geolocator = Nominatim(user_agent="teste_geo_cia")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# df_cia['geolocalizacao'] = df_cia['Endereço'].apply(geocode)
# df_cia['geolocalizacao']
geocode('primasea Candeias')

In [6]:
geolocator = GoogleV3(api_key=chave)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
geocode('primasea Candeias')

Location(Via Candeiras Km 6, s/n - Distrito Industrial, Candeias - BA, 43813-000, Brazil, (-12.7049802, -38.4720619, 0.0))

# Teste com os dois
Fazer teste nessa ordem através de funções: 
- Nome Fantasia + Endereco (sem bairro) + Cidade + UF
- Razão Social + Endereco (sem bairro) + Cidade + UF
- Nome Fantasia + Cidade + UF
- Razão Social + Cidade + UF
- Endereço + Bairro + Cidade + UF
- CEP > Endereço 

Salvar fonte de cada um.

Primeiro para o Nominatim. Se faltarem muitos depois mapquest e por ultimo google.

In [42]:

df_cia['geolocalizacao'] = None
df_cia['nome_fantasia'] = df_cia['nome_fantasia'].fillna('')
df_cia['Busca'] = None
df_cia['latitude'] = None
df_cia['longitude'] = None

geolocator_nomatim = Nominatim(user_agent="teste_geo_cia")
geolocator_google = GoogleV3(api_key=chave)
def faz_geo_nome_fantasia(df_cia, geolocator_escolhido):
    geocode = RateLimiter(geolocator_escolhido.geocode, min_delay_seconds=1)
    for index, row in df_cia.iterrows():
        geolocalizacao = row['geolocalizacao']
        if row['nome_fantasia'] == '':
            continue
        
        if geolocalizacao is None:
            busca = row['nome_fantasia'] + ' ' + row['mun_descricao']
            geolocalizacao = geocode(busca)

            if geolocalizacao is None:
                continue
            else:
                df_cia.loc[index, 'geolocalizacao'] = geolocalizacao[0]
                df_cia.loc[index, 'Busca'] = busca
                df_cia.loc[index, 'latitude'] = geolocalizacao.latitude
                df_cia.loc[index, 'longitude'] = geolocalizacao.longitude
        # elif geolocalizacao is None:
        #     busca = row['nome_fantasia'] + ' ' + row['end_logradouro'] + ' ' + row['mun_descricao']
        #     geolocalizacao = geocode(busca)
        #     df_cia.loc[index, 'geolocalizacao'] = geolocalizacao[0]
        #     df_cia.loc[index, 'Busca'] = busca

    return df_cia


df_cia_teste_nome_fantasia = faz_geo_nome_fantasia(df_cia, geolocator_nomatim)
df_cia_teste_nome_fantasia = faz_geo_nome_fantasia(df_cia_teste_nome_fantasia, geolocator_google)

df_cia_teste_nome_fantasia.to_excel('dados/dados_cia_teste_nome_fantasia.xlsx', index=False)

df_cia_teste_nome_fantasia

,cnpj,razao_social,nome_fantasia,end_logradouro,end_numero,end_complemento,end_bairro,mun_descricao,end_uf,end_cep,...,principal_executivo,cargo,cadastro,atualizacao,Endereço,Endereço_sem_bairro,geolocalizacao,Busca,latitude,longitude
0,02.119.258/0002-18,Fertipar Fertilizantes do Nordeste Ltda,Fertipar,Via Matoim,S/N,CIA Norte,Distrito Industrial,Candeias,BA,43813000,...,Alceu Feldmann,Presidente,2000-08-21 00:00:00,2023-03-21 00:00:00,"Via Matoim, S/N, CIA Norte, Distrito Industria...","Via Matoim, S/N, CIA Norte, Candeias, BA","Via Matoim, S/N, Cia Norte - Distrito Industri...",Fertipar Candeias,-12.712677,-38.475512
1,01.002.047/0003-19,Plumatex Colchoes Industrial Ltda,Plumatex,Rod BA-522 Km 01,S/N,,Distrito Industrial,Candeias,BA,43813000,...,Rafael Miguel de Melo,Diretor Comercial,2000-08-22 00:00:00,2022-10-14 00:00:00,"Rod BA-522 Km 01, S/N, Distrito Industrial, Ca...","Rod BA-522 Km 01, S/N, Candeias, BA","BA-531, 1 - Caroba, Candeias - BA, Brazil",Plumatex Candeias,-12.702502,-38.474126
2,61.156.501/0086-45,Mosaic Fertilizantes do Brasil Ltda,Mosaic,Rod BA-522 Km 01,S/N,Cx Postal 22,Distrito Industrial,Candeias,BA,43813000,...,Floris Eduard Alexsander Bielders,Presidente,2000-08-22 00:00:00,2022-09-19 00:00:00,"Rod BA-522 Km 01, S/N, Cx Postal 22, Distrito ...","Rod BA-522 Km 01, S/N, Cx Postal 22, Candeias, BA",None,None,None,None
3,27.515.154/0012-25,Proquigel Quimica S/A,Policarbonatos,Via Matoim,S/N,Fazenda Caroba,Distrito Industrial,Candeias,BA,43813000,...,Roberto Noronha Santos,Presidente,2000-10-04 00:00:00,2023-11-22 00:00:00,"Via Matoim, S/N, Fazenda Caroba, Distrito Indu...","Via Matoim, S/N, Fazenda Caroba, Candeias, BA",None,None,None,None
4,02.402.478/0001-73,Unigel Plasticos S/A,Unigel,Via Matoim,S/N,Fazenda Caroba,Distrito Industrial,Candeias,BA,43813000,...,Roberto Noronha Santos,Presidente,2000-10-04 00:00:00,2023-07-14 00:00:00,"Via Matoim, S/N, Fazenda Caroba, Distrito Indu...","Via Matoim, S/N, Fazenda Caroba, Candeias, BA","Unigel, Vila Caroba, Candeias, Região Geográfi...",Unigel Candeias,-12.703322,-38.489229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,43.505.539/0001-05,Vedamax Industria Ltda,Vedamax Industria,Rua Vivaldo Paim 1,1090,,CIA Sul,Simoes Filho,BA,43700000,...,Lucas Luan da Silva Vieira,Socio Administrador,2024-01-29 00:00:00,NaN,"Rua Vivaldo Paim 1, 1090, CIA Sul, Simoes Filh...","Rua Vivaldo Paim 1, 1090, Simoes Filho, BA","Simoes Filho, Simões Filho - State of Bahia, B...",Vedamax Industria Simoes Filho,-12.786276,-38.402154
186,09.410.326/0001-61,DMS Industria e Comercio de Refrigeracao Ltda,DMS Engenharia,Rua Cidade de Belmonte,834,,CIA Sul,Simoes Filho,BA,43700000,...,Deny Monteiro Fernandes,Socio Administrador,2024-02-15 00:00:00,NaN,"Rua Cidade de Belmonte, 834, CIA Sul, Simoes F...","Rua Cidade de Belmonte, 834, Simoes Filho, BA","R. Cidade de Belmonte, Simões Filho - BA, 4370...",DMS Engenharia Simoes Filho,-12.830026,-38.392712
187,29.336.297/0001-42,Petronor Industria e Comercio Ltda,Maxlimp,Via Penetracao C,773,,CIA Sul,Simoes Filho,BA,43700000,...,Claudio Almeida de Brito,Socio Administrador,2024-02-22 00:00:00,NaN,"Via Penetracao C, 773, CIA Sul, Simoes Filho, BA","Via Penetracao C, 773, Simoes Filho, BA","Simoes Filho, Simões Filho - State of Bahia, B...",Maxlimp Simoes Filho,-12.786276,-38.402154
188,21.436.882/0001-02,Carbongreen Tecnologia Ambiental Ltda,Carbongreen,Via de Penetracao IV,798,Setor 01,CIA Sul,Simoes Filho,BA,43700000,...,Thiago Barbosa Lima,Diretor Financeiro,2022-10-11 00:00:00,NaN,"Via de Penetracao IV, 798, Setor 01, CIA Sul, ...","Via de Penetracao IV, 798, Setor 01, Simoes Fi...","Simões Filho - State of Bahia, 43700-000, Brazil",Carbongreen Simoes Filho,-12.834865,-38.437556


In [40]:

geolocator_nomatim = Nominatim(user_agent="teste_geo_cia")
geocode_nomatim = RateLimiter(geolocator_nomatim.geocode, min_delay_seconds=1)
geolocator_google = GoogleV3(api_key=chave)
geocode_google = RateLimiter(geolocator_google.geocode, min_delay_seconds=1)

# geocode_nomatim('mosaic fertilizantes Candeias')
geocode_google('Mosaic Fertilizantes Candeias')

Location(Candeias - State of Bahia, Brazil, (-12.6726284, -38.5431855, 0.0))

In [44]:
df_cia_teste_nome_fantasia['geolocalizacao'].isna().sum()

17